# Face detection and recognition training pipeline

The following example illustrates how to fine-tune an InceptionResnetV1 model on your own dataset. This will mostly follow standard pytorch training patterns.

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

In [ ]:
!pip install facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 119.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Define run parameters

The dataset should follow the VGGFace2/ImageNet-style directory layout. Modify `data_dir` to the location of the dataset on wish to finetune on.

In [ ]:
#data_dir = '../data/test_images'

batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 8

#### Determine if an nvidia GPU is available

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


#### Define MTCNN module

See `help(MTCNN)` for more details.

In [ ]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

#### Perfom MTCNN facial detection

Iterate through the DataLoader object and obtain cropped faces.

In [ ]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]

loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')

# Remove mtcnn to reduce GPU memory usage
del mtcnn

In [ ]:
dataset = datasets.ImageFolder('/content/drive/MyDrive/face train')

#### Define Inception Resnet V1 module

See `help(InceptionResnetV1)` for more details.

In [ ]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

#### Define optimizer, scheduler, dataset, and dataloader

In [ ]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder('/content/drive/MyDrive/face train', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

#### Define loss and evaluation functions

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

#### Train model

In [ ]:
# (修改後的版本)

writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

# <<< 新增開始 (1/3) >>>
# 初始化一個變數來追蹤最佳的驗證損失
best_val_loss = float('inf')
# 定義權重檔案的儲存路徑和名稱
save_path = '/content/drive/MyDrive/best_model_weights.pt'
# <<< 新增結束 (1/3) >>>

print('\n\nInitial')
print('-' * 10)
resnet.eval()
# 注意：facenet-pytorch 的 training.pass_epoch 會回傳 (loss, metrics)
# 我們在初始評估時不需要用到回傳值，所以可以忽略
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    # 訓練階段
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    # 驗證階段
    # <<< 修改開始 (2/3) >>>
    # 接收 pass_epoch 的回傳值，我們需要驗證損失 (validation loss)
    val_loss, _ = training.pass_epoch(
    # <<< 修改結束 (2/3) >>>
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    # <<< 新增開始 (3/3) >>>
    # 檢查當前的驗證損失是否比之前記錄的最好損失還要低
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        # 儲存模型的 state_dict (只包含模型的可訓練參數，是推薦的做法)
        torch.save(resnet.state_dict(), save_path)
        print(f"  -> Validation loss decreased ({best_val_loss:.4f}). Saving model to {save_path}")
    # <<< 新增結束 (3/3) >>>

writer.close()
print("\nTraining complete!")
print(f"The best model weights are saved at: {save_path}")



Initial
----------
Valid |     3/3    | loss:    0.0001 | fps:  303.1371 | acc:    1.0000   

Epoch 1/8
----------
Train |    10/10   | loss:    0.0192 | fps:  203.7556 | acc:    0.9937   
Valid |     3/3    | loss:    0.0002 | fps:  386.5256 | acc:    1.0000   
  -> Validation loss decreased (0.0002). Saving model to /content/drive/MyDrive/best_model_weights.pt

Epoch 2/8
----------
Train |    10/10   | loss:    0.0030 | fps:  197.6884 | acc:    1.0000   
Valid |     3/3    | loss:    0.0001 | fps:  425.9232 | acc:    1.0000   
  -> Validation loss decreased (0.0001). Saving model to /content/drive/MyDrive/best_model_weights.pt

Epoch 3/8
----------
Train |    10/10   | loss:    0.0004 | fps:  188.3873 | acc:    1.0000   
Valid |     3/3    | loss:    0.0001 | fps:  287.2452 | acc:    1.0000   

Epoch 4/8
----------
Train |    10/10   | loss:    0.0003 | fps:  200.0564 | acc:    1.0000   
Valid |     3/3    | loss:    0.0001 | fps:  429.1811 | acc:    1.0000   

Epoch 5/8
----------

In [ ]:
import torch
from torchvision import transforms
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import os
from tqdm import tqdm

# --- 1. 參數設定 (請根據您的情況修改) ---

# 權重檔案的路徑 (您在訓練時儲存的檔案)
WEIGHTS_PATH = '/content/drive/MyDrive/best_model_weights.pt'

# 測試圖片所在的資料夾路徑
TEST_IMAGE_DIR = '/content/drive/MyDrive/face test/test' # 請務必修改為您的測試圖片資料夾

# 您在訓練時的類別名稱
# ImageFolder 會按照字母順序對類別進行編號。
# 您需要提供一個列表，其索引要與訓練時的 class_to_idx 對應。
# 您可以在訓練腳本的輸出中找到 `training_dataset.class_to_idx`
# 例如: {'person_A': 0, 'person_B': 1} -> CLASS_NAMES = ['person_A', 'person_B']
# 如果您只有一個類別，就像這樣:
CLASS_NAMES = ['personB','train'] # <--- 請務必修改為您訓練時的類別名稱

# 設備設定
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"正在使用裝置: {DEVICE}")


# --- 2. 載入模型 ---

# 建立模型架構
# 確保 num_classes 與您訓練時的設定完全相同
model = InceptionResnetV1(
    classify=True,
    num_classes=len(CLASS_NAMES)
)

# 載入您訓練好的權重
model.load_state_dict(torch.load(WEIGHTS_PATH, map_location=DEVICE))

# 將模型設定為評估模式 (eval mode)
# 這會關閉 Dropout 等只在訓練時使用的層
model.eval()

# 將模型送到指定裝置
model.to(DEVICE)

# 初始化 MTCNN 用於臉部偵測
mtcnn = MTCNN(
    image_size=160,
    margin=0,
    min_face_size=20,
    device=DEVICE,
    keep_all=True # 偵測圖片中的所有臉部
)


# --- 3. 定義圖片預處理 ---
# 必須和訓練時的驗證集 (validation set) 使用完全相同的轉換
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])


# --- 4. 執行預測 ---

# 確保測試資料夾存在
if not os.path.isdir(TEST_IMAGE_DIR):
    print(f"錯誤：測試資料夾 '{TEST_IMAGE_DIR}' 不存在。")
else:
    # 遍歷資料夾中的所有檔案
    image_files = [f for f in os.listdir(TEST_IMAGE_DIR) if os.path.isfile(os.path.join(TEST_IMAGE_DIR, f))]

    for image_name in tqdm(image_files, desc="正在處理測試圖片"):
        image_path = os.path.join(TEST_IMAGE_DIR, image_name)

        try:
            # 載入圖片
            img = Image.open(image_path).convert('RGB')

            # 使用 MTCNN 偵測臉部
            # mtcnn 會直接回傳裁切好並縮放成 160x160 的臉部 Tensor
            faces_tensors = mtcnn(img)

            if faces_tensors is None:
                print(f"\n在圖片 '{image_name}' 中未偵測到臉部。")
                continue

            print(f"\n--- 圖片: {image_name} (偵測到 {len(faces_tensors)} 張臉) ---")

            # 將臉部 tensors 送到指定裝置
            faces_tensors = faces_tensors.to(DEVICE)

            # 對偵測到的每一張臉進行預測
            with torch.no_grad(): # 在推論時不需要計算梯度
                # 進行預測
                outputs = model(faces_tensors)
                # 將模型的輸出 (logits) 轉換為機率
                probabilities = torch.softmax(outputs, dim=1)

            # 處理每一張臉的預測結果
            for i, prob in enumerate(probabilities):
                # 找到機率最高的類別索引
                top_prob, top_idx = torch.max(prob, 0)
                pred_class_name = CLASS_NAMES[top_idx.item()]
                confidence = top_prob.item()

                print(f"  臉部 #{i+1}: 預測為 '{pred_class_name}'，信心度: {confidence:.2%}")

        except Exception as e:
            print(f"\n處理圖片 '{image_name}' 時發生錯誤: {e}")

print("\n測試完成。")

正在使用裝置: cuda:0


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/best_model_weights.pt'

In [ ]:
print(dataset.class_to_idx)

{'personB': 0, 'train': 1}


In [ ]:
import cv2
import torch
from facenet_pytorch import MTCNN
from PIL import Image
import os
from tqdm import tqdm

# --- 參數設定 ---

# 輸入影片的路徑
VIDEO_PATH = "personB.mkv"

# 輸出的根目錄
OUTPUT_ROOT_DIR = "/content/drive/MyDrive/test other"

# 裁切後臉部圖片的尺寸
IMAGE_SIZE = 160

# 在臉部周圍增加的邊距 (pixels)
# 這有助於包含整個頭部，而不僅僅是臉部
MARGIN = 40

# 每隔多少幀處理一次，設定為 1 表示處理每一幀
# 如果影片很長，可以增加此數值以加快速度 (例如 5 或 10)
FRAME_SKIP = 15

# --- 程式碼主體 ---

def extract_faces_from_video(video_path, output_root, image_size, margin, frame_skip):
    """
    從影片中偵測、裁切並儲存臉部。

    :param video_path: 輸入影片檔案的路徑。
    :param output_root: 儲存臉部資料集的根目錄。
    :param image_size: 輸出臉部圖片的尺寸。
    :param margin: 臉部邊界框的邊距。
    :param frame_skip: 處理幀的間隔。
    """
    # 檢查 GPU 是否可用，如果可用則使用 GPU 加速
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f'正在使用裝置: {device}')

    # 初始化 MTCNN 模型
    # keep_all=True 表示偵測畫面中的「所有」臉部
    mtcnn = MTCNN(
        image_size=image_size,
        margin=margin,
        min_face_size=20,
        thresholds=[0.6, 0.7, 0.7],
        factor=0.709,
        post_process=True,
        device=device,
        keep_all=True  # 非常重要：確保偵測所有臉部
    )

    # 建立輸出目錄
    if not os.path.exists(output_root):
        os.makedirs(output_root)

    # 根據影片名稱建立子目錄 (身份/類別目錄)
    video_filename = os.path.basename(video_path)
    identity_name = os.path.splitext(video_filename)[0]
    identity_output_path = os.path.join(output_root, identity_name)

    if not os.path.exists(identity_output_path):
        os.makedirs(identity_output_path)

    print(f"所有偵測到的臉部將儲存至: {identity_output_path}")

    # 打開影片檔案
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"錯誤: 無法開啟影片檔案 {video_path}")
        return

    # 獲取影片總幀數以用於進度條
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_count = 0
    face_count = 0

    with tqdm(total=total_frames, desc="處理影片中") as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break # 影片結束

            # 更新進度條
            pbar.update(1)
            frame_count += 1

            # 根據 frame_skip 決定是否處理當前幀
            if frame_count % frame_skip != 0:
                continue

            # OpenCV 讀取的是 BGR 格式，需轉換為 MTCNN 需要的 RGB 格式
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(frame_rgb)

            # 使用 MTCNN 偵測臉部，返回邊界框 (bounding boxes) 和信心度
            boxes, probs = mtcnn.detect(img_pil)

            # 如果偵測到臉部
            if boxes is not None:
                for i, box in enumerate(boxes):
                    # MTCNN 模組有內建的裁切和儲存功能，但為了更高自訂性，我們手動處理
                    # 這裡的 box 已經是加上 margin 後的結果，但我們自己來做更保險

                    x1, y1, x2, y2 = box

                    # 確保邊界框不會超出圖片範圍
                    x1, y1 = max(0, x1), max(0, y1)
                    x2, y2 = min(img_pil.width - 1, x2), min(img_pil.height - 1, y2)

                    # 檢查裁切框是否有效
                    if x1 >= x2 or y1 >= y2:
                        continue

                    # 從原始 PIL 圖像中裁切臉部
                    face_pil = img_pil.crop((x1, y1, x2, y2))

                    # 將臉部縮放至目標尺寸
                    face_resized = face_pil.resize((image_size, image_size), Image.Resampling.LANCZOS)

                    # 建立儲存路徑和檔名
                    save_path = os.path.join(identity_output_path, f"{face_count:06d}.png")

                    # 儲存圖片
                    face_resized.save(save_path)

                    face_count += 1

    # 釋放資源
    cap.release()
    print("\n處理完成！")
    print(f"總共處理了 {frame_count} 幀。")
    print(f"成功偵測並儲存了 {face_count} 張臉部圖片。")

if __name__ == '__main__':
    extract_faces_from_video(
        video_path=VIDEO_PATH,
        output_root=OUTPUT_ROOT_DIR,
        image_size=IMAGE_SIZE,
        margin=MARGIN,
        frame_skip=FRAME_SKIP
    )

正在使用裝置: cuda:0
所有偵測到的臉部將儲存至: /content/drive/MyDrive/test other/personB


處理影片中: 100%|██████████| 1353/1353 [00:19<00:00, 70.24it/s]


處理完成！
總共處理了 1353 幀。
成功偵測並儲存了 90 張臉部圖片。


In [ ]:
!pip install onnx onnx-tf
# 確保你的 tensorflow 版本是兼容的，onnx-tf 通常需要 tensorflow 2.x

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 53.2 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.2
    Uninstalling typeguard-4.4.2:
      Successfully uninstalled typeguard-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
import torch
from facenet_pytorch import InceptionResnetV1
import torch.nn as nn

# --- 參數設定 ---
PYTORCH_WEIGHTS_PATH = '/content/drive/MyDrive/best_model_weights.pth'
ONNX_MODEL_PATH = '/content/drive/MyDrive/facenet_model.onnx'
DEVICE = torch.device('cpu')

# <<< 解決方案的核心改動 >>>

# 1. 建立一個與您的權重檔案「完全匹配」的模型架構
print("正在載入 PyTorch 模型 (匹配權重檔案的架構)...")

# 獲取您訓練時的類別數量 (從錯誤訊息中得知是 2)
NUM_CLASSES_IN_CHECKPOINT = 2

# 建立模型，並確保 `num_classes` 與權重檔案中的完全一致
model = InceptionResnetV1(
    pretrained='vggface2',        # 從 VGGFace2 開始，以獲得骨幹網路的初始權重
    classify=True,                # 確保模型有分類層
    num_classes=NUM_CLASSES_IN_CHECKPOINT # 關鍵！設定為您微調時的類別數
)

# 2. 載入您微調過的權重 (這次使用 strict=True，因為架構完全匹配)
model.load_state_dict(torch.load(PYTORCH_WEIGHTS_PATH, map_location=DEVICE))
print("權重成功載入到完整模型中。")

# 3. 移除分類頭，只保留骨幹網路
# 我們將最後的 logits 層替換為一個什麼都不做的 Identity 層
# 這樣模型的前向傳播就會在倒數第二層停止，這正是我們想要的特徵向量
model.logits = nn.Identity()

# 將模型設定為評估模式
model.eval()
model.to(DEVICE)
print("分類頭已移除，模型準備好進行特徵提取。")


# --- 匯出為 ONNX 格式 (這部分不變) ---
print(f"準備將模型匯出為 ONNX 格式至: {ONNX_MODEL_PATH}")

dummy_input = torch.randn(1, 3, 160, 160, device=DEVICE)
input_names = ["input_1"]
output_names = ["output_1"]

try:
    torch.onnx.export(
        model,
        dummy_input,
        ONNX_MODEL_PATH,
        verbose=True,
        input_names=input_names,
        output_names=output_names,
        opset_version=11,
        dynamic_axes={
            'input_1': {0: 'batch_size'},
            'output_1': {0: 'batch_size'}
        }
    )
    print("\n✅ 模型成功匯出為 ONNX 格式！")

except Exception as e:
    print(f"\n❌ 匯出 ONNX 時發生錯誤: {e}")

正在載入 PyTorch 模型 (匹配權重檔案的架構)...
權重成功載入到完整模型中。
分類頭已移除，模型準備好進行特徵提取。
準備將模型匯出為 ONNX 格式至: /content/drive/MyDrive/facenet_model.onnx

✅ 模型成功匯出為 ONNX 格式！


In [ ]:
import onnx
from onnx_tf.backend import prepare

# --- 參數設定 ---
ONNX_MODEL_PATH = '/content/drive/MyDrive/facenet_model.onnx'
TF_SAVEDMODEL_DIR = '/content/drive/MyDrive/facenet_tf_savedmodel' # 輸出資料夾路徑

# --- 執行轉換 ---
print(f"正在從 {ONNX_MODEL_PATH} 轉換為 TensorFlow SavedModel...")

# 載入 ONNX 模型
onnx_model = onnx.load(ONNX_MODEL_PATH)

# 準備轉換器
# `prepare` 函式會將 ONNX 圖譜轉換為 TensorFlow 圖譜
tf_rep = prepare(onnx_model)

# 匯出為 SavedModel 格式
tf_rep.export_graph(TF_SAVEDMODEL_DIR)

print(f"✅ TensorFlow SavedModel 已成功儲存至: {TF_SAVEDMODEL_DIR}")

/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.18.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're u

ModuleNotFoundError: No module named 'keras.src.engine'

In [ ]:
import tensorflow as tf

# --- 參數設定 ---
TF_SAVEDMODEL_DIR = '/content/drive/MyDrive/facenet_tf_savedmodel'
TFLITE_MODEL_PATH = '/content/drive/MyDrive/facenet_model.tflite' # 最終的 TFLite 檔案路徑

# --- 執行轉換 ---
print(f"正在從 {TF_SAVEDMODEL_DIR} 轉換為 TFLite...")

# 建立 TFLite 轉換器
converter = tf.lite.TFLiteConverter.from_saved_model(TF_SAVEDMODEL_DIR)

# (可選) 啟用優化，例如量化 (Quantization)
# 這會減小模型大小並可能加速推論，但可能會稍微犧牲精度
# converter.optimizations = [tf.lite.Optimize.DEFAULT]

# 執行轉換
tflite_model = converter.convert()

# 將轉換後的模型寫入檔案
with open(TFLITE_MODEL_PATH, 'wb') as f:
    f.write(tflite_model)

print(f"✅ TFLite 模型已成功儲存至: {TFLITE_MODEL_PATH}")

In [ ]:
!pip install onnx2keras

  Preparing metadata (setup.py) ... done
  Created wheel for onnx2keras: filename=onnx2keras-0.0.24-py3-none-any.whl size=24576 sha256=04f367284bfb99d0db08f06dad141ced472b8d08b22b2c5df16596a15151b635
  Stored in directory: /root/.cache/pip/wheels/a5/a0/fb/eb5b60162f43fa1c199234d7dcfc976fa929715ff513615be8
Successfully built onnx2keras


In [ ]:
import onnx
from onnx2keras import onnx_to_keras
import tensorflow as tf
import numpy as np
import sys

# --- 參數設定 ---
ONNX_MODEL_PATH = '/content/drive/MyDrive/facenet_model.onnx'
TFLITE_MODEL_PATH = '/content/drive/MyDrive/facenet_model.tflite'

print(f"正在從 {ONNX_MODEL_PATH} 載入 ONNX 模型...")
onnx_model = onnx.load(ONNX_MODEL_PATH)


# <<< 解決方案的核心：清理 ONNX 節點名稱 >>>
print("\n正在清理 ONNX 模型中的節點名稱 (替換 '/' 為 '_')...")
for i, node in enumerate(onnx_model.graph.node):
    # 清理節點本身的名稱
    node.name = node.name.replace('/', '_')
    # 清理節點的輸入和輸出張量的名稱
    for j, output in enumerate(node.output):
        node.output[j] = output.replace('/', '_')
    for k, input in enumerate(node.input):
        node.input[k] = input.replace('/', '_')
print("✅ 節點名稱清理完成。")
# <<< 清理步驟結束 >>>


input_names = [input.name.replace('/', '_') for input in onnx_model.graph.input]
print(f"清理後的 ONNX 輸入層名稱: {input_names}")


# --- 核心轉換步驟 ---
print("\n正在將清理後的 ONNX 轉換為 Keras 模型...")
keras_model = None
try:
    keras_model = onnx_to_keras(
        onnx_model,
        input_names,
        change_ordering=True
    )
    print("✅ ONNX 成功轉換為 Keras 模型。")
    keras_model.summary()
except Exception as e:
    print(f"❌ 轉換為 Keras 模型時出錯: {e}")
    print("詳細錯誤追蹤:")
    import traceback
    traceback.print_exc() # 打印更詳細的錯誤追蹤
    sys.exit()

# --- 將 Keras 模型轉換為 TFLite ---
if keras_model:
    print(f"\n正在將 Keras 模型轉換為 TFLite 至: {TFLITE_MODEL_PATH}")

    try:
        converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
        # (可選) 優化
        # converter.optimizations = [tf.lite.Optimize.DEFAULT]
        tflite_model_content = converter.convert()

        with open(TFLITE_MODEL_PATH, 'wb') as f:
            f.write(tflite_model_content)

        print(f"✅ TFLite 模型已成功儲存至: {TFLITE_MODEL_PATH}")

    except Exception as e:
        print(f"❌ 從 Keras 轉換到 TFLite 時發生錯誤: {e}")
        import traceback
        traceback.print_exc()

正在從 /content/drive/MyDrive/facenet_model.onnx 載入 ONNX 模型...

正在清理 ONNX 模型中的節點名稱 (替換 '/' 為 '_')...
✅ 節點名稱清理完成。
清理後的 ONNX 輸入層名稱: ['input_1']

正在將清理後的 ONNX 轉換為 Keras 模型...


Traceback (most recent call last):
  File "<ipython-input-49-72e53485130e>", line 37, in <cell line: 0>
    keras_model = onnx_to_keras(
                  ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/onnx2keras/converter.py", line 175, in onnx_to_keras
    AVAILABLE_CONVERTERS[node_type](
  File "/usr/local/lib/python3.11/dist-packages/onnx2keras/convolution_layers.py", line 164, in convert_conv
    conv = keras.layers.Conv2D(
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/conv2d.py", line 109, in __init__
    super().__init__(
  File "/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py", line 107, in __init__
    super().__init__(activity_regularizer=activity_regularizer, **kwargs)
  File "/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py", line 287, in __init__
    raise ValueError(
ValueError: Unrecognized keyword arguments passed to Conv2D: {'weights': [

❌ 轉換為 Keras 模型時出錯: Unrecognized keyword arguments passed to Conv2D: {'weights': [array([[[[ 1.46863014e-01, -4.76438910e-01,  5.61059006e-02,
          -2.14676723e-01, -4.93226200e-01,  5.60793281e-01,
           2.67437428e-01, -5.64456403e-01,  1.57014787e-01,
           9.36317742e-01,  2.22712681e-01, -3.03584546e-01,
          -2.72456825e-01,  4.21023965e-01,  2.29700565e-01,
           2.35679731e-01,  5.33283539e-02,  1.07005909e-01,
           4.85131860e-01, -1.33772242e+00, -4.25311066e-02,
           3.29831123e-01, -9.33961987e-01,  4.76854503e-01,
          -3.29307705e-01,  2.11264789e-01, -3.51134807e-01,
          -3.06206942e-01, -8.62144411e-01,  7.35618114e-01,
           2.69898213e-02,  2.65381727e-02],
         [ 5.86803332e-02, -5.59451222e-01,  1.43837452e-01,
          -5.20780265e-01, -8.95169437e-01,  8.36856484e-01,
           1.49539873e-01,  1.95476785e-01,  1.07321814e-01,
          -5.72025418e-01, -1.22431979e-01, -5.95490262e-02,
          -2.3168319

TypeError: object of type 'NoneType' has no len()